## My Neural Network

In [10]:
import numpy as np
import scipy as sc

import matplotlib.pyplot as plt

## Datasets

In [55]:
input_dataset = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
output_dataset = np.array([[0], [1], [1], [0]])

## Neural Network Layer class

In [56]:
np.random.seed(5)

In [73]:
class NeuralLayer:
    def __init__(self, num_connections_entering, num_neurons, activation_fn):
        self.activation_fn = activation_fn
        
        # create from -1 to 1
        self.bias = np.random.rand(1, num_neurons) * 2 - 1
        self.weights = np.random.rand(num_connections_entering, num_neurons) * 2 - 1

## Activation functions

In [72]:
# function, derivate
sigmoid = (
    lambda x : 1 / (1 + np.e ** (-x)),
    lambda x : x * (1 - x)
)

In [65]:
## Cost function

In [75]:
# function, derivate
mean_square_error = (
    lambda prediction, real : np.mean((prediction - real) ** 2),
    lambda prediction, real : (prediction - real)
)

In [76]:
def createNeuralNetwork():
    NeuralNetwork = [
        NeuralLayer(num_connections_entering=2, num_neurons=3, activation_fn=sigmoid),
        NeuralLayer(num_connections_entering=2, num_neurons=3, activation_fn=sigmoid),
    ]
    
    return NeuralNetwork
    
XOR = createNeuralNetwork()
print(XOR)

[<__main__.NeuralLayer object at 0x10e8922d0>, <__main__.NeuralLayer object at 0x10e892350>]


In [63]:
def train(neural_network, input, output, cost_function, learning_rate = 0.2):
    # Foward
    input_to_layer = input
    
    for layer in neural_network:
        pondate_sum = input_to_layer @ layer.weights + layer.bias
        input_to_layer = after_activation = layer.activation_fn(pondate_sum)
    
    prediction = after_activation
    
    
    # Calculate error
    
    # Backpropagation
    
    
        

SyntaxError: unexpected EOF while parsing (<ipython-input-63-fc5bc86a88c8>, line 9)